## Notebook 2: Model + LightningModule + Logging

Objective: Implement model architectures (Custom CNN, ResNet), encapsulate them
           in a PyTorch Lightning LightningModule, and set up comprehensive
           logging for training and evaluation.

In [ ]:

# --- Imports ---
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import pytorch_lightning as pl
from torch.optim import AdamW # Example optimizer
from torch.optim.lr_scheduler import ReduceLROnPlateau # Example scheduler
# Add metrics imports later if needed (e.g., from torchmetrics)
# --- Imports ---
# ... (existing imports) ...
import torchmetrics # Make sure torchmetrics is installed (pip install torchmetrics)

# Specific metrics (adjust average strategy as needed)
from torchmetrics.classification import MulticlassAccuracy, MulticlassPrecision, MulticlassRecall, MulticlassF1Score, MulticlassAUROC, ConfusionMatrix

# --- Configuration ---
# These might be loaded from a config file or CLI args in a real script
NUM_CLASSES = 7 # From our NSCLC dataset
LEARNING_RATE = 1e-4
# ... other relevant configurations ...

## 🧠 Introduction to Histology Image Modeling

Histology image classification involves training models to recognize patterns
in tissue samples, often stained (e.g., H&E). Key challenges include:
- **Large Image Size:** Whole Slide Images (WSIs) are massive, requiring patch-based approaches.
- **Subtle Features:** Distinguishing between classes often relies on fine-grained details.
- **Stain Variability:** Differences in staining protocols can affect image appearance.
- **Class Imbalance:** Some tissue types might be much rarer than others.

**Common Approaches:**
- **Binary Classification:** Distinguishing between two states (e.g., tumor vs. normal).
- **Multi-Class Classification:** Assigning patches to one of several predefined categories (like our 7 region types).
- **Transfer Learning:** Leveraging models pre-trained on large datasets (like ImageNet) and fine-tuning them for the specific histology task (often beneficial).
- **Custom Architectures:** Designing CNNs tailored to the specific characteristics of histology data.

This section focuses on implementing a custom CNN as a baseline.

In [15]:
class SimpleHistologyCNN(nn.Module):
    """
    A basic Convolutional Neural Network for histology patch classification.
    """
    def __init__(self, num_classes: int):
        super().__init__()
        # Define convolutional layers
        # Example: (Input channels, Output channels, Kernel size, Stride, Padding)
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2) # Reduces spatial dims by half

        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.gap = nn.AdaptiveAvgPool2d((1, 1)) # Global Average Pooling

        # Placeholder for calculating the flattened size dynamically
        # We'll need to pass a dummy input through the conv layers once
        self._feature_size = None
        self._calculate_feature_size() # Calculate size during init

        # # Define fully connected layers
        # self.fc1 = nn.Linear(self._feature_size, 128)
        # self.relu4 = nn.ReLU()
        # self.dropout = nn.Dropout(0.5) # Dropout for regularization
        # self.fc2 = nn.Linear(128, num_classes)

    def _calculate_feature_size(self):
        """Helper to determine the size of the flattened features after conv layers."""
        # Create a dummy input matching expected dimensions (Batch, Channels, Height, Width)
        # Assuming input patches are, for example, 256x256
        # Adjust the dummy input size if your patches are different!
        dummy_input = torch.randn(1, 3, 256, 256) # Example size
        x = self.pool1(self.relu1(self.conv1(dummy_input)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = self.pool3(self.relu3(self.conv3(x)))
        # Flatten the output
        self._feature_size = x.view(x.size(0), -1).shape[1]
        print(f"Calculated feature size after conv layers: {self._feature_size}")


    def forward(self, x):
        # Convolutional part
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = self.pool3(self.relu3(self.conv3(x)))
        # Global Average Pooling
        x = self.gap(x)
        # Flatten the output
        x = x.view(x.size(0), -1)
        return x

## ⚡ LightningModule: Encapsulating Model & Training Logic

A `pytorch_lightning.LightningModule` organizes the PyTorch code related to training,
validation, and testing. It encapsulates:
- The model architecture (our CNN or ResNet).
- The forward pass logic.
- The calculation of the loss.
- The code performed during training, validation, and test steps.
- The configuration of optimizers and learning rate schedulers.

This makes the code cleaner, more reusable, and integrates seamlessly with the
Lightning `Trainer`.

In [ ]:
class HistologyClassifier(pl.LightningModule):
    """
    PyTorch Lightning Module for histology image classification.
    """
    def __init__(self,
                 encoder_name: str = 'resnet18', # Or 'custom_cnn'
                 num_classes: int = NUM_CLASSES,
                 learning_rate: float = LEARNING_RATE,
                 optimizer: str = 'AdamW',
                 lr_scheduler: str = 'ReduceLROnPlateau',
                 pretrained: bool = True): # Added pretrained flag
        super().__init__()
        # Save hyperparameters for logging and potential loading later
        # Important: Don't save the actual model instance here if it's large or complex
        self.save_hyperparameters()


        if encoder_name == 'resnet18':
            self.encoder = models.resnet18(pretrained=pretrained) # Load pretrained ResNet18
            # remove the fully connected, and give the number of channels of the output layer
            num_ftrs = self.encoder.fc.in_features
            self.encoder.fc = nn.Identity() # Remove the final fully connected layer
        elif encoder_name == 'custom_cnn':
            self.encoder = SimpleHistologyCNN(num_classes=num_classes)
            # For custom CNN, we need to define the final fully connected layer
            num_ftrs = self.encoder._feature_size # Get the calculated feature size
        else:
            raise ValueError(f"Unsupported encoder: {encoder_name}")

        ##########################
        ## WRITE YOUR CODE HERE ##
        ##########################
        self.fc = ...  # Adjust for number of classes

        # Loss function (CrossEntropyLoss is common for multi-class classification)
        self.criterion = nn.CrossEntropyLoss()

        # Example placeholder for metrics (we'll add proper ones in Task 4.3)
        # from torchmetrics.classification import MulticlassAccuracy
        # self.accuracy = MulticlassAccuracy(num_classes=num_classes)
        metric_args = {'num_classes': num_classes, 'average': 'macro'}

        self.train_metrics = torchmetrics.MetricCollection({
            ##########################
            ## WRITE YOUR CODE HERE ##
            ##########################
            # AUC requires probabilities or logits
            # 'auroc': MulticlassAUROC(**metric_args, thresholds=None) # Use default thresholds
        })
        self.val_metrics = self.train_metrics.clone(prefix='val_') # Clone with prefix
        self.test_metrics = self.train_metrics.clone(prefix='test_') # Clone with prefix

        # Separate AUROC metric as it requires logits/probabilities directly
        # auroc_args = {'num_classes': num_classes, 'average': 'macro', 'thresholds': None}
        # self.train_auroc = MulticlassAUROC(**auroc_args)
        # self.val_auroc = MulticlassAUROC(**auroc_args)
        # self.test_auroc = MulticlassAUROC(**auroc_args)

    def forward(self, x):
        # The forward pass is simply delegated to the underlying model
        x = self.encoder(x)
        x = self.fc(x)
        return x

    def _calculate_loss(self, batch, batch_idx, stage='train'):
        images, labels = batch
        logits = self.forward(images)
        loss = self.criterion(logits, labels)

        # --- Log Loss ---
        self.log(f'{stage}_loss_step', loss, on_step=True, on_epoch=False, prog_bar=False, logger=True)
        self.log(f'{stage}_loss_epoch', loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)

        # --- Calculate & Log Metrics ---
        preds = torch.argmax(logits, dim=1)
        # Apply softmax to get probabilities for AUROC
        probs = F.softmax(logits, dim=1)

        # Select the correct metric collection based on the stage
        if stage == 'train':
            metrics_obj = self.train_metrics
            # auroc_obj = self.train_auroc
        elif stage == 'val':
            metrics_obj = self.val_metrics
            # auroc_obj = self.val_auroc
        elif stage == 'test':
            metrics_obj = self.test_metrics
            # auroc_obj = self.test_auroc
        else:
             raise ValueError(f"Invalid stage: {stage}")

        # Update metrics
        metrics_obj.update(preds, labels)
        # auroc_obj.update(probs, labels) # AUROC uses probabilities/logits

        # Log metrics at the end of the epoch
        self.log_dict(metrics_obj, on_step=False, on_epoch=True, logger=True)
        # self.log(f'{stage}_auroc', auroc_obj, on_step=False, on_epoch=True, logger=True) # Log AUROC separately

        return loss

    def training_step(self, batch, batch_idx):
        # Defines the logic for a single training step
        return self._calculate_loss(batch, batch_idx, stage='train')

    def validation_step(self, batch, batch_idx):
        # Defines the logic for a single validation step
        # This is used to monitor performance on unseen data during training
        return self._calculate_loss(batch, batch_idx, stage='val')

    def test_step(self, batch, batch_idx):
        # Defines the logic for a single testing step
        # This is used after training to evaluate the final model performance
        return self._calculate_loss(batch, batch_idx, stage='test')

    def configure_optimizers(self):
        """
        Choose optimizers and learning-rate schedulers to use during training.
        """
        # If pretrained is True, freeze encoder parameters (do not train encoder)
        if self.hparams.pretrained:
            for param in self.encoder.parameters():
                param.requires_grad = False

        # Collect parameters to optimize (only trainable ones)
        params_to_optimize = (
            list(filter(lambda p: p.requires_grad, self.encoder.parameters())) +
            list(self.global_avg_pool.parameters()) +
            list(self.flatten.parameters()) +
            list(self.fc.parameters())
        )

        if self.hparams.optimizer.lower() == 'adamw':
            optimizer = AdamW(
            params_to_optimize,
            lr=self.hparams.learning_rate
            )
        elif self.hparams.optimizer.lower() == 'adam':
            optimizer = torch.optim.Adam(
            params_to_optimize,
            lr=self.hparams.learning_rate
            )
        # Add other optimizers like SGD if needed
        else:
            raise ValueError(f"Unsupported optimizer: {self.hparams.optimizer}")

        # Configure learning rate scheduler if specified
        if self.hparams.lr_scheduler.lower() == 'reducelronplateau':
            # Reduce learning rate when validation loss plateaus
            scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)
            return {
                'optimizer': optimizer,
                'lr_scheduler': {
                    'scheduler': scheduler,
                    'monitor': 'val_loss_epoch', # Metric to monitor
                    'interval': 'epoch',
                    'frequency': 1
                }
            }
        elif self.hparams.lr_scheduler.lower() == 'none':
            return optimizer # Return only the optimizer if no scheduler
        else:
             raise ValueError(f"Unsupported lr_scheduler: {self.hparams.lr_scheduler}")



### Metrics for Evaluation

Choosing appropriate metrics is crucial for evaluating classification performance,
especially in medical imaging where class imbalance can be common. We use:

- **Accuracy:** Overall percentage of correct predictions. Can be misleading if classes are imbalanced.
- **Precision:** Of the samples predicted as positive, how many actually were? (TP / (TP + FP)). High precision means fewer false positives. Important when the cost of a false positive is high.
- **Recall (Sensitivity):** Of the actual positive samples, how many were correctly identified? (TP / (TP + FN)). High recall means fewer false negatives. Important when the cost of missing a positive case is high.
- **F1-Score:** The harmonic mean of precision and recall (2 * (Precision * Recall) / (Precision + Recall)). Provides a balance between precision and recall.
- **AUC (Area Under the ROC Curve):** Measures the model's ability to distinguish between classes across all possible thresholds. An AUC of 1.0 is perfect, 0.5 is random guessing. Often a good overall measure, especially for imbalanced datasets.

We use `'macro'` averaging, which calculates the metric independently for each class and then takes the unweighted average. This treats all classes equally, regardless of their size. `'weighted'` averaging considers class support.


In [17]:
plModel = HistologyClassifier(
    encoder_name='resnet18',
    num_classes=NUM_CLASSES,
    learning_rate=LEARNING_RATE,
    optimizer='AdamW',
    lr_scheduler='ReduceLROnPlateau',
    pretrained=True
)


/home/mbenimam/Documents/work/neubias/venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/mbenimam/Documents/work/neubias/venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [19]:
# simulate batch with labels
batch = (torch.randn(8, 3, 256, 256), torch.randint(0, NUM_CLASSES, (8,)))
# Example training step
loss = plModel.training_step(batch, 0)
print(f"Training step loss: {loss.item()}")
# Example validation step
val_loss = plModel.validation_step(batch, 0)
print(f"Validation step loss: {val_loss.item()}")
# Example test step
test_loss = plModel.test_step(batch, 0)
print(f"Test step loss: {test_loss.item()}")



/home/mbenimam/Documents/work/neubias/venv/lib/python3.10/site-packages/pytorch_lightning/core/module.py:441: You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet. This is most likely because the model hasn't been passed to the `Trainer`


Training step loss: 2.183617353439331
Validation step loss: 2.183617353439331
Test step loss: 2.183617353439331
